In [1]:
from datasets import load_metric
metric = load_metric("rouge")

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [2]:
import argparse
from datetime import datetime
import os
import time

import numpy as np
from transformers import GPT2LMHeadModel,AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tnrange, tqdm

from dataset import GPT21024Dataset 
from utils import add_special_tokens, generate_sample, generate_beam_sample, set_seed


In [3]:
root_dir = './gpt2_1024_data'
ids_file = './CNN/ids.json'
valid_data = GPT21024Dataset(root_dir, ids_file, mode='valid', length=500)  #validation on only 500 datasets

In [4]:
train_data = GPT21024Dataset(root_dir,ids_file,mode='train',length=3000) #training on only 3000 datasets
valid_data = GPT21024Dataset(root_dir,ids_file,mode='valid',length=500)  #validation on only 500 datasets
tokenizer = add_special_tokens()
ignore_idx = tokenizer.pad_token_id
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 1024)

In [5]:
from transformers import GPT2Tokenizer
def add_special_tokens():
	""" Returns GPT2 tokenizer after adding separator and padding tokens """
	tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
	special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
	num_add_toks = tokenizer.add_special_tokens(special_tokens)
	return tokenizer

tokenizer = add_special_tokens()

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model.resize_token_embeddings(len(tokenizer))
device = torch.device('cuda:4')
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [13]:
calc_rouge_scores(cands, refs)

{'rouge1': 0.6, 'rouge2': 0.0, 'rougeL': 0.6, 'rougeLsum': 0.6}

In [8]:
len(valid_data)

500

In [12]:
cands[:10]

['\ned............,,..ism.',
 ' Theically.ically.icallyically.ically.icallyicallyically.icallyicallyicallyised.......ised.ism',
 '!',
 '!',
 ' the',
 ' Clark',
 ' I',
 'I',
 '\n',
 ' The']

In [11]:
refs[:10]

['Kaleb Kula punched across face as he stood alone in a driveway. Middle school pupils cheered and filmed attack to put on Facebook. Parents say he has been target of bullies at Elkton Middle School. Police say they have charged juvenile with second degree assault. <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>',
 "Ahmed Fareed strangled Major Janet Gilson with a rope. She was found hidden under a sofa three days after being reported missing. Fareed was the ex-husband of Maj Gilson's niece. <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|> <|pad|>

In [10]:
refs, cands = generate_beam_sample(valid_data, tokenizer, model, num=len(valid_data), device=device)

/notebook/greenAI/summarization_test/utils.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  next_token_probs = F.softmax(next_token_logits)


  0%|          | 0/99 [00:00<?, ?it/s]

/notebook/greenAI/summarization_test/utils.py:108: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  next_token_probs = F.softmax(next_token_logits)


1 1


  0%|          | 0/99 [00:00<?, ?it/s]

...

  0%|          | 0/99 [00:00<?, ?it/s]

144 144


  0%|          | 0/99 [00:00<?, ?it/s]

145 145


  0%|          | 0/99 [00:00<?, ?it/s]

In [ ]:
cands = []
refs = []
for i in range(len(valid_data)):
    sample = valid_data[i]
    idx = sample['sum_idx']
    context = sample['article'][:idx].tolist()
    summary = sample['article'][idx+1:][:100].tolist()
    print(tokenizer.decode(context), end='\n\n')
    print('actual_summary', end='\n\n')
    print(tokenizer.decode(summary), end='\n\n')
    refs.append(tokenizer.decode(summary))
    print ("generated")
    ref, cand = generate_beam_sample([sample], tokenizer, model, num=1, device=device)
    cands.append(cand)
    refs.append(ref)
    if (i % 100 == 0):
        print("cand", cand, end='\n\n')
        print("ref", ref, end='\n\n')

In [14]:
print(tokenizer.decode(context), end='\n\n')
print('actual_summary', end='\n\n')
print(tokenizer.decode(summary), end='\n\n')

At 95, an elderly man from Denmark is still able to lift weights that most people a quarter of his age wouldn't be able to budge from the ground. Even more astonishingly, it was only two-and-a-half years ago that powerlifter Svend Stensgaard was rushed to hospital after having a heart attack. Yet a video filmed of him working out at the gym - in a room full of boys young enough to be his great-grandchildren - proves just how fit and healthy he is in later life. In the footage, the jacked Dane defies stereotypes of pensioners as frail and feeble beings clad in their bedroom slippers. Both standing and lying on his back, he is seen shifting a whopping 290 pounds of weight as he controls his breathing to establish a rhythm. Mr Stensgaard says in the interview that exercising, which releases a lot of stress-fighting endorphins, is comparable to a ` dosage of morphine'for him. Power pa : Svend Stensgaard, 95, showcases his impressive strength while powerlifting at his local gym in Denmark. 

In [4]:
!pip3 install rouge_score

  Created wheel for rouge-score: filename=rouge_score-0.1.2-cp37-none-any.whl size=24957 sha256=a6aa3daec5aefd4eac50166f9629a5441663a0fd4095b4f26f809f6e75e2b7cb
  Stored in directory: /root/.cache/pip/wheels/df/aa/59/74f33db3bbedf322bcaadca4a43750ea5eb523bbe742d78b25
Successfully built rouge-score
ERROR: deeppavlov 0.6.1 requires fastapi==0.38.1, which is not installed.
ERROR: deeppavlov 0.6.1 requires flasgger==0.9.2, which is not installed.
ERROR: deeppavlov 0.6.1 requires fuzzywuzzy==0.17.0, which is not installed.
ERROR: deeppavlov 0.6.1 requires pymorphy2==0.8, which is not installed.
ERROR: deeppavlov 0.6.1 requires pymorphy2-dicts-ru, which is not installed.
ERROR: deeppavlov 0.6.1 requires pyopenssl==19.0.0, which is not installed.
ERROR: deeppavlov 0.6.1 requires pytelegrambotapi==3.6.6, which is not installed.
ERROR: deeppavlov 0.6.1 requires rusenttokenize==0.0.5, which is not installed.
ERROR: deeppavlov 0.6.1 requires uvicorn==0.9.0, which is not installed.
ERROR: tensorfl